<a href="https://colab.research.google.com/github/brownian-explorer/securities-master/blob/main/dataupdater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Securities Master Database
Aim of the code here is to maintain a database of OHLC data from different sources. Will explore tick level data in the future. Storage in google drive in csv format

In [1]:
import yfinance as yf
import pandas as pd
from google.colab import drive
import os
from datetime import datetime
import pytz
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [51]:
class YahooDownloader:
  def __init__(self):
    self.storage_folder = "/content/drive/MyDrive/findata/OHLC_yahoo"
    self.log_file = "/content/drive/MyDrive/findata/log/log.csv"
    self.exchange_file = "/content/drive/MyDrive/findata/trading_hours.csv"

  # Function to load existing tickers from log.csv
  def load_existing_tickers(self):
      if not os.path.exists(self.log_file):
          return dict()
      existing_tickers = dict()
      with open(self.log_file, 'r') as file:
          lines = file.readlines()
          for line in lines[1:]:  # Skip the header line
              ticker, last_updated = line.strip().split(',')
              existing_tickers[ticker] = datetime.strptime(last_updated.strip(), '%Y-%m-%d')
      return existing_tickers #Return tickers: last_updated date in dictionary form

  # Function to update log.csv with new tickers and new updated dates
  def update_log(self, new_ticker, last_updated):
      with open(self.log_file, 'a') as file:
          file.write(f"{new_ticker},{last_updated}\n")

  #Remove duplicates from log file, as update_log will create duplicate entries
  def remove_duplicates_from_log(self):
      if not os.path.exists(self.log_file):
          return
      log_data = {}
      with open(self.log_file, 'r') as file:
          lines = file.readlines()
          for line in lines[1:]:
              ticker, last_updated = line.strip().split(',')
              log_data.setdefault(ticker, []).append(last_updated)

      with open(self.log_file, 'w') as file:
          file.write("Ticker,YahooLastUpdated\n")
          for ticker, dates in log_data.items():
              latest_date = max(dates)
              file.write(f"{ticker}, {latest_date}\n")

  def split_old_new(self, long_ticker_list):
    existing_tickers = self.load_existing_tickers().keys()
    new_tickers = [ticker for ticker in long_ticker_list if ticker not in existing_tickers]
    return new_tickers

  def split_tickers(self, ticker_list, n):
    for i in range(0, len(ticker_list), n):
      yield ticker_list[i:i + n]

  def convert_timezone(self, date, time):
    datetime_str = date.split()[0] + " " + time

    # Define the format of the datetime string
    datetime_format = "%Y-%m-%d %I:%M %p"

    # Convert the combined string to a datetime object
    datetime_obj = datetime.strptime(datetime_str, datetime_format)
    return datetime_obj

  # Function to fetch and store stock price data from yahoo finance, and add the open, close time of the security
  def fetch_yahoo_data(self, ticker_df):
      trading_hours_df = pd.read_csv(self.exchange_file)
      existing_tickers = self.load_existing_tickers()
      new_tickers = self.split_old_new(list(ticker_df["long_ticker"]))
      existing_tickers_split = list(self.split_tickers(list(existing_tickers.keys()), 500))
      if len(new_tickers) > 0:
        new_tickers_split = list(self.split_tickers(new_tickers, 500))
      else:
        new_tickers_split = []
      failed = []
      # print(new_tickers_split)
      # print(existing_tickers_split)
      # return

      if len(new_tickers_split) > 0:
        for ticker_group in new_tickers_split:
          filtered = ticker_df[ticker_df['long_ticker'].isin(ticker_group)]
          yahoo_tickers = filtered["Ticker_yahoo"]
          exchange = filtered["exchange"]

          #Fetch data from yfinance
          bulk_df = yf.download(" ".join(yahoo_tickers), end=datetime.today().strftime('%Y-%m-%d'), group_by="ticker")

          for ticker, exchange in zip(list(bulk_df.columns.levels[0]),exchange):
            df = bulk_df[ticker].dropna()
            long_ticker = ticker_df.query('Ticker_yahoo == @ticker')["long_ticker"].iloc[0]
            break
            if not df.empty:
              exchange_info = trading_hours_df[trading_hours_df["Symbol"] == exchange]
              df = df.reset_index()
              df["Date_Open"] = df.reset_index()["Date"].apply(lambda x: self.convert_timezone(str(x), str(exchange_info["Open"].iloc[0])))
              df["Date_Close"] = df.reset_index()["Date"].apply(lambda x: self.convert_timezone(str(x), str(exchange_info["Close"].iloc[0])))
              df = df.set_index("Date")

              # Group data by year
              data_by_year = df.groupby(df.index.year)

              # Create a folder for each ticker
              ticker_directory = os.path.join(self.storage_folder, ticker[0], long_ticker)
              os.makedirs(ticker_directory, exist_ok=True)

              # Save data into separate CSV files for each year in the ticker folder
              for year, year_data in data_by_year:
                year_file_name = os.path.join(ticker_directory, f"{year}.csv")
                year_data.to_csv(year_file_name)

                # Update the log file
                self.update_log(long_ticker, datetime.today().strftime('%Y-%m-%d'))
                print(f"Data for {long_ticker} saved to findata folder")
            else:
              failed.append(long_ticker)

      if len(existing_tickers_split) > 0:
        for ticker_group in existing_tickers_split:
          filtered = ticker_df[ticker_df['long_ticker'].isin(ticker_group)]
          yahoo_tickers = filtered["Ticker_yahoo"]
          exchange = filtered["exchange"]
          start_date = f"{list(existing_tickers.values())[0].year}-01-01"

          try:
            bulk_df = yf.download(" ".join(yahoo_tickers), start=start_date, group_by="ticker")
          except:
            failed.append(yahoo_tickers)

          for ticker, exchange in zip(list(bulk_df.columns.levels[0]),exchange):
            long_ticker = ticker_df.query('Ticker_yahoo == @ticker')["long_ticker"].iloc[0]
            print(f"Data for {long_ticker} already exists. Updating...")

            df = bulk_df[ticker]
            if not df.empty:
              exchange_info = trading_hours_df[trading_hours_df["Symbol"] == exchange]
              df = df.reset_index()
              df["Date_Open"] = df.reset_index()["Date"].apply(lambda x: self.convert_timezone(str(x), str(exchange_info["Open"].iloc[0])))
              df["Date_Close"] = df.reset_index()["Date"].apply(lambda x: self.convert_timezone(str(x), str(exchange_info["Close"].iloc[0])))
              df = df.set_index("Date")

              # Group data by year
              data_by_year = df.groupby(df.index.year)

              # Create a folder for each ticker
              ticker_directory = os.path.join(self.storage_folder, ticker[0], long_ticker)
              os.makedirs(ticker_directory, exist_ok=True)

              # Save data into separate CSV files for each year in the ticker folder
              for year, year_data in data_by_year:
                year_file_name = os.path.join(ticker_directory, f"{year}.csv")
                year_data.to_csv(year_file_name)

              # Update the log file
              self.update_log(long_ticker, datetime.today().strftime('%Y-%m-%d'))
              print(f"Data for {ticker} updated from {start_date} to today.")
            else:
              failed.append(long_ticker)
      pd.DataFrame(failed).T.to_csv("/content/drive/MyDrive/findata/failed.csv")
      self.remove_duplicates_from_log()

In [52]:
csv_file_path = "/content/drive/MyDrive/findata/ticker_master.csv"

if not os.path.exists(csv_file_path):
    print("CSV file not found.")
else:
  # Read tickers from the CSV file
  ticker_df = pd.read_csv(csv_file_path, keep_default_na=False, na_values=['_'])[["long_ticker","exchange","Ticker_yahoo"]]

yahoo_update = YahooDownloader()

In [53]:
yahoo_update.fetch_yahoo_data(ticker_df)

[*********************100%%**********************]  36 of 36 completed

ERROR:yfinance:
36 Failed downloads:
ERROR:yfinance:['TNONW', 'WAVSW', 'VERBW', 'THWWW', 'VGASW', 'VMCAW', 'THCPW', 'WKSPW', 'VRMEW', 'VIEWW', 'UKOMW', 'VSACW', 'XPDBW', 'WESTW', 'UHGWW', 'USGOW', 'TWLVW', 'VCXAW', 'ZIVOW', 'YOTAW', 'TGAAW', 'ZAPPW', 'WTMAR', 'YSBPW', 'ZLSWW', 'ZURAW', 'VWEWW', 'TLGYW', 'TRKAW', 'VIIAW', 'XOSWW', 'TMTCR', 'WLDSW', 'WWACW', 'WGSWW', 'WINVW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



[*********************100%%**********************]  500 of 500 completed
Data for NASDAQ:AACG already exists. Updating...
Data for AACG updated from 2023-01-01 to today.
Data for NASDAQ:AACI already exists. Updating...
Data for AACI updated from 2023-01-01 to today.
Data for NASDAQ:AACIU already exists. Updating...
Data for AACIU updated from 2023-01-01 to today.
Data for NASDAQ:AACIW already exists. Updating...
Data for AACIW updated from 2023-01-01 to today.
Data for NASDAQ:AADI already exists. Updating...
Data for AADI updated from 2023-01-01 to today.
Data for NASDAQ:AAL already exists. Updating...
Data for AAL updated from 2023-01-01 to today.
Data for NASDAQ:AAME already exists. Updating...
Data for AAME updated from 2023-01-01 to today.
Data for NASDAQ:AAOI already exists. Updating...
Data for AAOI updated from 2023-01-01 to today.
Data for NASDAQ:AAON already exists. Updating...
Data for AAON updated from 2023-01-01 to today.
Data for NASDAQ:AAPL already exists. Updating...
Da

ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['EFHTR', 'ESHAR', 'ENERR', 'DMAQR', 'DISTR']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



Data for NASDAQ:CROX already exists. Updating...
Data for CROX updated from 2023-01-01 to today.
Data for NASDAQ:CRSP already exists. Updating...
Data for CRSP updated from 2023-01-01 to today.
Data for NASDAQ:CRSR already exists. Updating...
Data for CRSR updated from 2023-01-01 to today.
Data for NASDAQ:CRTO already exists. Updating...
Data for CRTO updated from 2023-01-01 to today.
Data for NASDAQ:CRUS already exists. Updating...
Data for CRUS updated from 2023-01-01 to today.
Data for NASDAQ:CRVL already exists. Updating...
Data for CRVL updated from 2023-01-01 to today.
Data for NASDAQ:CRVO already exists. Updating...
Data for CRVO updated from 2023-01-01 to today.
Data for NASDAQ:CRVS already exists. Updating...
Data for CRVS updated from 2023-01-01 to today.
Data for NASDAQ:CRWD already exists. Updating...
Data for CRWD updated from 2023-01-01 to today.
Data for NASDAQ:CRWS already exists. Updating...
Data for CRWS updated from 2023-01-01 to today.
Data for NASDAQ:CSBR already 

...


Data for NASDAQ:INZY already exists. Updating...
Data for INZY updated from 2023-01-01 to today.
Data for NASDAQ:IOAC already exists. Updating...
Data for IOAC updated from 2023-01-01 to today.
Data for NASDAQ:IOACU already exists. Updating...
Data for IOACU updated from 2023-01-01 to today.
Data for NASDAQ:IOACW already exists. Updating...
Data for IOACW updated from 2023-01-01 to today.
Data for NASDAQ:IOBT already exists. Updating...
Data for IOBT updated from 2023-01-01 to today.
Data for NASDAQ:IONM already exists. Updating...
Data for IONM updated from 2023-01-01 to today.
Data for NASDAQ:IONR already exists. Updating...
Data for IONR updated from 2023-01-01 to today.
Data for NASDAQ:IONS already exists. Updating...
Data for IONS updated from 2023-01-01 to today.
Data for NASDAQ:IOSP already exists. Updating...
Data for IOSP updated from 2023-01-01 to today.
Data for NASDAQ:IOVA already exists. Updating...
Data for IOVA updated from 2023-01-01 to today.
Data for NASDAQ:IPA alrea

ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['NVACR', 'MSSAR', 'NOVVR', 'NNAGR']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:['NETD']: Exception('%ticker%: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=%ticker%, cluster=finance]]')



Data for NASDAQ:MNDO already exists. Updating...
Data for MNDO updated from 2023-01-01 to today.
Data for NASDAQ:MNDY already exists. Updating...
Data for MNDY updated from 2023-01-01 to today.
Data for NASDAQ:MNKD already exists. Updating...
Data for MNKD updated from 2023-01-01 to today.
Data for NASDAQ:MNMD already exists. Updating...
Data for MNMD updated from 2023-01-01 to today.
Data for NASDAQ:MNOV already exists. Updating...
Data for MNOV updated from 2023-01-01 to today.
Data for NASDAQ:MNPR already exists. Updating...
Data for MNPR updated from 2023-01-01 to today.
Data for NASDAQ:MNRO already exists. Updating...
Data for MNRO updated from 2023-01-01 to today.
Data for NASDAQ:MNSB already exists. Updating...
Data for MNSB updated from 2023-01-01 to today.
Data for NASDAQ:MNSBP already exists. Updating...
Data for MNSBP updated from 2023-01-01 to today.
Data for NASDAQ:MNST already exists. Updating...
Data for MNST updated from 2023-01-01 to today.
Data for NASDAQ:MNTK alread

ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['STRCW', 'PLTNR', 'PPHPR', 'QOMOR']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:['XGN']: Exception('%ticker%: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=%ticker%, cluster=finance]]')



Data for NASDAQ:PFTAW already exists. Updating...
Data for PFTAW updated from 2023-01-01 to today.
Data for NASDAQ:PHUNW already exists. Updating...
Data for PHUNW updated from 2023-01-01 to today.
Data for NASDAQ:PIIIW already exists. Updating...
Data for PIIIW updated from 2023-01-01 to today.
Data for NASDAQ:PLAOW already exists. Updating...
Data for PLAOW updated from 2023-01-01 to today.
Data for NASDAQ:PLMIW already exists. Updating...
Data for PLMIW updated from 2023-01-01 to today.
Data for NASDAQ:PLTNR already exists. Updating...
Data for PLTNR updated from 2023-01-01 to today.
Data for NASDAQ:PLTNW already exists. Updating...
Data for PLTNW updated from 2023-01-01 to today.
Data for NASDAQ:PPHPR already exists. Updating...
Data for PPHPR updated from 2023-01-01 to today.
Data for NASDAQ:PPYAW already exists. Updating...
Data for PPYAW updated from 2023-01-01 to today.
Data for NASDAQ:PRENW already exists. Updating...
Data for PRENW updated from 2023-01-01 to today.
Data for 